In [3]:
import numpy as np 
import random 

In [23]:
#tic-tac-toe game class   
class TicTacToe:  
    #initialization 
    def __init__(self):  
        self.board = np.zeros(9) 
        self.done = False 
        self.winner = None  

    #loops through board, output is an array with empty cells 
    def available_positions(self): 
        cells = [] 
        for i in range(9): 
            if self.board[i] == 0: 
                cells.append(i) 
        return cells 

    #if an element is 0, that cell is open for player to make a move 
    def make_move(self, position, player): 
        if self.board[position] == 0: 
            self.board[position] = player 
            self.check_winner() 
            return True  
            #place player at available position and check if game is over 
        #not possible to place player here
        return False        

    #loop through board and check if the game is over 
    def check_winner(self):  
        #possible ways to win the game 
        combos = [
            [0, 1, 2], [3, 4, 5], [6, 7, 8], #rows 
            [0, 3, 6], [1, 4, 7], [2, 5, 8], #columns 
            [0, 4, 8], [2, 4, 6] #diagonals 
        ]  
        
        for combo in combos:  
            #add elements which are in the possible winning positions 
            s = self.board[combo[0]] + self.board[combo[1]] + self.board[combo[2]]
            #x has won the game 
            if s == 3: 
                self.done = True 
                self.winner = 1 
                return  
            #o has won the game 
            elif s == -3: 
                self.done = True 
                self.winner = -1 
                return 
        #the board is full but no one has won - draw 
        if 0 not in self.board: 
            self.done = True 
            self.winner = 0 

In [25]:
#q-learning agent 
class QLearningAgent: 
    def __init__(self, alpha=0.3, gamma=0.9, epsilon=0.2): 
        self.q = {} #key - (board state, next move), value - q_value (how good the move is)
        self.alpha = alpha 
        self.gamma = gamma 
        self.epsilon = epsilon 

    #input - state, action to be taken & output - q_val (default - 0) 
    def getQ(self, state, action): 
        return self.q.get((tuple(state), action), 0.0) 

    #what action to take - random/ based on what has been learnt so far 
    def choose_action(self, state, moves): 
        #random move 
        if random.random() < self.epsilon: 
            return random.choice(moves)  

        #loops through possible moves and finds one with the highest q value 
        qs = [] 
        for move in moves: 
            qs.append(self.getQ(state, move)) 
        maxQ = max(qs) 

        #for the highest q_val, what are the corresponding moves 
        best_moves = [] 
        for i in range(len(moves)): 
            move = moves[i] 
            q_val = qs[i] 
            if q_val == maxQ: 
                best_moves.append(move) 

        return random.choice(best_moves)  

    #updates itself after every move 
    def learn(self, state, action, reward, next_state, next_moves): 
        old_q = self.getQ(state, action) 
        if next_moves: 
            next_qs = [] 
            for move in next_moves: 
                next_qs.append(self.getQ(next_state, move)) 
            max_next_q = max(next_qs) 
        else: 
            max_next_q = 0 
        #new knowledge = old knowledge + (learning rate * [what actually happened - what was predicted])
        new_q = old_q + self.alpha*(reward + self.gamma*max_next_q - old_q) 
        self.q[(tuple(state), action)] = new_q 

In [27]:
#training function  
def train(agent_X, agent_O, episodes=50000): 
    for _ in range(episodes): 
        game = TicTacToe() 

        #X always starts 
        player = 1  
        while not game.done: 
            state = game.board.copy() 
            moves = game.available_positions() 
            if player == 1: 
                action = agent_X.choose_action(state, moves) 
            else: 
                action = agent_O.choose_action(state, moves)  
            game.make_move(action, player) 
            next_state = game.board.copy() 
            next_moves = game.available_positions() 

            if game.done: 
                #draw 
                if game.winner == 0: 
                    reward_X = 0.5 
                    reward_O = 0.5 
                #X wins 
                elif game.winner == 1: 
                    reward_X = 1 
                    reward_O = 0  
                #O wins 
                else: 
                    reward_X = 0 
                    reward_Y = 1 
            else: 
                reward_X = 0 
                reward_O = 0 

            #updating q values of X and O  
            agent_X.learn(state, action, reward_X, next_state, next_moves) 
            agent_O.learn(state, action, reward_O, next_state, next_moves) 

            #switching out the player 
            player = player*-1 

In [29]:
#play function   
def play(agent): 
    game = TicTacToe() 
    #X starts 
    player = 1  
    #game board 
    print("positions:\n0|1|2\n3|4|5\n6|7|8\n") 

    while not game.done: 
        print(game.board.reshape(3, 3)) 
        if player == 1: 
            move = int(input("enter your move - 0 to 8")) 
        else: 
            move = agent.choose_action(game.board, game.available_positions()) 
        game.make_move(move, player) 
        player = player*-1 

    #board when game is done 
    print(game.board.reshape(3, 3))  

    if game.winner == 1: 
        print("you won") 
    elif game.winner == -1: 
        print("ai wins") 
    else: 
        print("draw") 

In [39]:
#actually training the agents   
agent_X = QLearningAgent() 
agent_O = QLearningAgent() 

print("training in progress") 
train(agent_X, agent_O, episodes=200000) 
print("training complete")  
agent_O.epsilon = 0 

training in progress
training complete


In [33]:
#actually playing with O   
play(agent_O)

positions:
0|1|2
3|4|5
6|7|8

[[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]


enter your move - 0 to 8 4


[[0. 0. 0.]
 [0. 1. 0.]
 [0. 0. 0.]]
[[ 0.  0.  0.]
 [ 0.  1.  0.]
 [ 0.  0. -1.]]


enter your move - 0 to 8 2


[[ 0.  0.  1.]
 [ 0.  1.  0.]
 [ 0.  0. -1.]]
[[ 0.  0.  1.]
 [ 0.  1. -1.]
 [ 0.  0. -1.]]


enter your move - 0 to 8 6


[[ 0.  0.  1.]
 [ 0.  1. -1.]
 [ 1.  0. -1.]]
you won


In [35]:
play(agent_O)

positions:
0|1|2
3|4|5
6|7|8

[[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]


enter your move - 0 to 8 0


[[1. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]
[[ 1. -1.  0.]
 [ 0.  0.  0.]
 [ 0.  0.  0.]]


enter your move - 0 to 8 4


[[ 1. -1.  0.]
 [ 0.  1.  0.]
 [ 0.  0.  0.]]
[[ 1. -1.  0.]
 [-1.  1.  0.]
 [ 0.  0.  0.]]


enter your move - 0 to 8 8


[[ 1. -1.  0.]
 [-1.  1.  0.]
 [ 0.  0.  1.]]
you won


In [37]:
play(agent_O)

positions:
0|1|2
3|4|5
6|7|8

[[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]


enter your move - 0 to 8 1


[[0. 1. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]
[[ 0.  1.  0.]
 [ 0.  0. -1.]
 [ 0.  0.  0.]]


enter your move - 0 to 8 0


[[ 1.  1.  0.]
 [ 0.  0. -1.]
 [ 0.  0.  0.]]
[[ 1.  1.  0.]
 [ 0.  0. -1.]
 [ 0.  0. -1.]]


enter your move - 0 to 8 3


[[ 1.  1.  0.]
 [ 1.  0. -1.]
 [ 0.  0. -1.]]
[[ 1.  1.  0.]
 [ 1.  0. -1.]
 [-1.  0. -1.]]


enter your move - 0 to 8 4


[[ 1.  1.  0.]
 [ 1.  1. -1.]
 [-1.  0. -1.]]
[[ 1.  1.  0.]
 [ 1.  1. -1.]
 [-1. -1. -1.]]
ai wins


In [41]:
play(agent_O) 

positions:
0|1|2
3|4|5
6|7|8

[[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]


enter your move - 0 to 8 2


[[0. 0. 1.]
 [0. 0. 0.]
 [0. 0. 0.]]
[[ 0. -1.  1.]
 [ 0.  0.  0.]
 [ 0.  0.  0.]]


enter your move - 0 to 8 4


[[ 0. -1.  1.]
 [ 0.  1.  0.]
 [ 0.  0.  0.]]
[[ 0. -1.  1.]
 [ 0.  1.  0.]
 [-1.  0.  0.]]


...

[[ 0. -1.  1.]
 [ 0.  1.  0.]
 [-1.  1.  0.]]
[[ 0. -1.  1.]
 [-1.  1.  0.]
 [-1.  1.  0.]]


enter your move - 0 to 8 8


[[ 0. -1.  1.]
 [-1.  1.  0.]
 [-1.  1.  1.]]
[[ 0. -1.  1.]
 [-1.  1. -1.]
 [-1.  1.  1.]]


enter your move - 0 to 8 0


[[ 1. -1.  1.]
 [-1.  1. -1.]
 [-1.  1.  1.]]
you won
